In [19]:
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import networkx as nx
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.nn import GCNConv

# Load data
tweet_data = pd.read_csv(r'C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users_hate_glove.content', header=None, delimiter="\t")
tweet_data = tweet_data.rename(columns={0: "user_id", 301: "hate_label"})
tweet_data = tweet_data[tweet_data['hate_label'] != "other"]
tweet_data.reset_index(inplace=True)
tweet_data.drop(columns=['index'], inplace=True)
tweet_data_features = tweet_data.drop(columns=['hate_label'])
tweet_data_labels = tweet_data['hate_label']


tweet_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4971 entries, 0 to 4970
Columns: 302 entries, user_id to hate_label
dtypes: float64(300), int64(1), object(1)
memory usage: 11.5+ MB


In [18]:
import pandas as pd
import networkx as nx

edge_df = pd.read_csv(r"C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users.edges", header=None, names=['edges'])
user_ids = tweet_data['user_id'].tolist()



edge_df['source'] = edge_df['edges'].str.split(' ').str[0].astype(int)
edge_df['target'] = edge_df['edges'].str.split(' ').str[1].astype(int)
edge_df.drop(['edges'],axis=1,inplace=True)

edge_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2286592 entries, 0 to 2286591
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   source  int32
 1   target  int32
dtypes: int32(2)
memory usage: 17.4 MB


In [21]:
user_ids

[0,
 22,
 29,
 44,
 85,
 99,
 100,
 118,
 119,
 124,
 127,
 142,
 178,
 183,
 192,
 199,
 269,
 279,
 327,
 354,
 398,
 405,
 426,
 439,
 518,
 528,
 542,
 545,
 547,
 567,
 572,
 580,
 583,
 588,
 595,
 615,
 633,
 645,
 653,
 672,
 673,
 709,
 718,
 787,
 801,
 815,
 838,
 882,
 895,
 907,
 925,
 950,
 974,
 982,
 1035,
 1075,
 1108,
 1111,
 1214,
 1228,
 1251,
 1263,
 1281,
 1314,
 1346,
 1356,
 1368,
 1375,
 1400,
 1406,
 1456,
 1461,
 1476,
 1498,
 1520,
 1528,
 1554,
 1573,
 1607,
 1624,
 1679,
 1702,
 1723,
 1749,
 1751,
 1782,
 1826,
 1864,
 1938,
 1943,
 2038,
 2048,
 2057,
 2081,
 2131,
 2188,
 2200,
 2209,
 2227,
 2241,
 2243,
 2324,
 2365,
 2428,
 2456,
 2458,
 2460,
 2464,
 2493,
 2500,
 2516,
 2555,
 2579,
 2607,
 2608,
 2644,
 2657,
 2658,
 2674,
 2677,
 2706,
 2733,
 2801,
 2817,
 2827,
 2834,
 2881,
 2893,
 2899,
 2905,
 2949,
 2967,
 2990,
 3008,
 3011,
 3014,
 3017,
 3027,
 3038,
 3090,
 3094,
 3113,
 3130,
 3143,
 3154,
 3159,
 3161,
 3185,
 3198,
 3222,
 3245,
 326

In [22]:
filtered_edges = edge_df[edge_df['source'].isin(user_ids) & edge_df['target'].isin(user_ids)]
filtered_edges

,source,target
2161,83842,83842
2520,47490,47490
2524,42590,42590
2527,42590,15570
3323,73139,73139
...,...,...
2285534,96613,74857
2285836,74031,74031
2285946,24930,24930
2285951,22853,22853


In [23]:
G = nx.Graph()

# Add nodes to the graph
G.add_nodes_from(user_ids)

# Add edges to the graph
for index, row in filtered_edges.iterrows():
    source = row['source']
    target = row['target']
    if source in user_ids and target in user_ids:
        G.add_edge(source, target)



num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Print the number of nodes and edges
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 4971
Number of edges: 14591


In [25]:
adj_matrix = nx.adjacency_matrix(G, nodelist=user_ids)
adj_matrix = adj_matrix.toarray()

# Check the number of nodes in the graph
num_nodes = len(user_ids)

# Compare the shape of the adjacency matrix with the number of nodes
print("Shape of adjacency matrix:", adj_matrix.shape)
print("Number of nodes in the graph:", num_nodes)

Shape of adjacency matrix: (4971, 4971)
Number of nodes in the graph: 4971


In [ ]:
# # Initialize the model
# class GCN(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(input_dim, hidden_dim)
#         self.conv2 = GCNConv(hidden_dim, hidden_dim)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index
#         x = self.conv1(x, edge_index)
#         x = torch.relu(x)
#         x = self.conv2(x, edge_index)
#         x = torch.relu(x)
#         x = torch.mean(x, dim=0)  # Global pooling
#         x = torch.sigmoid(self.fc(x))
#         return x

# input_dim = X_train_tensor.shape[1]
# hidden_dim = 64
# output_dim = len(label_encoder.classes_)
# model = GCN(input_dim, hidden_dim, output_dim)

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# # Training the model
# model.train()
# for epoch in range(10):
#     optimizer.zero_grad()
#     outputs = model(train_data)  # Pass the whole data object
#     loss = criterion(outputs, train_data.y)
#     loss.backward()
#     optimizer.step()
#     print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# # Evaluation
# model.eval()
# with torch.no_grad():
#     test_outputs = model(test_data)
#     test_loss = criterion(test_outputs, test_data.y)
#     print(f'Test Loss: {test_loss.item()}')
